In [19]:
from uuid import uuid4
import pandas as pd
from langchain_core.documents import Document
import os
from model_config import VietnameseEmbeddings

In [20]:
from elasticsearch import Elasticsearch
# Kết nối tới Elasticsearch
es = Elasticsearch(
    "https://my-elasticsearch-project-fc9fd1.es.ap-southeast-1.aws.elastic.cloud:443",
    api_key="aXY4NkpKUUJaNVNfUEdnZHdVZ186MExmVk1iclZRWWlrS1hpeDRhOWRGUQ=="
)
index = "base"
embeddings = VietnameseEmbeddings()

In [37]:
excel = r"D:\DATN\QA_System\eval\testset\final_dataset_qa.xlsx"
chunks = pd.read_excel(excel)
chunks.head(2)

,metadata,text,question,answer
0,{'source': 'Chiến lược phát triển đại học Bách...,CHIẾN LƯỢC PHÁT TRIỂN\nTÓM LƯỢC CHIẾN LƯỢC PHÁ...,Mục tiêu chiến lược phát triển Đại học Bách kh...,Mục tiêu chiến lược phát triển Đại học Bách kh...
1,{'source': 'Chiến lược phát triển đại học Bách...,CHIẾN LƯỢC PHÁT TRIỂN\nTÓM LƯỢC CHIẾN LƯỢC PHÁ...,"Theo chiến lược phát triển, Đại học Bách khoa ...","Theo chiến lược phát triển, Đại học Bách khoa ..."


In [56]:
import json
import pandas as pd
import time
from tqdm import tqdm
from elasticsearch import Elasticsearch

def search(query, top_k=1):
    # Add delay to avoid API overload (adjust as needed)
    time.sleep(0.1)  # 100ms delay between requests
    
    embedding = embeddings.embed_query(query)
    result = es.search(
        index=index,
        knn={
            "field": "vector",
            "query_vector": embedding,
            "k": top_k,
            "num_candidates": top_k * 2
        },
        source_excludes=["vector"]
    )
    res = [hit['_source']["metadata"] for hit in result['hits']['hits']]
    return res

def evaluate(excel_path, top_k_values=[1, 5, 10, 20]):
    chunk = pd.read_excel(excel_path)
    total_questions = len(chunk)
    
    # Khởi tạo bộ đếm điểm
    hits = {k: 0 for k in top_k_values}
    mrr = {k: 0 for k in top_k_values}
    
    # Add progress bar
    progress_bar = tqdm(chunk.iterrows(), total=total_questions, desc="Evaluating questions")
    
    for _, row in progress_bar:
        question = row['question']
        metadata = row['metadata']
        if isinstance(metadata, str):
            metadata = metadata.replace("'", "\"")
            try:
                metadata = json.loads(metadata)
            except json.JSONDecodeError:
                continue
        correct_source = metadata
        
        # Update progress bar description with current question
        progress_bar.set_description(f"Processing: {question[:30]}...")
        
        # Thực hiện tìm kiếm
        search_results = search(question, top_k=max(top_k_values))
        
        # Kiểm tra kết quả và tính điểm
        sources = [result.get("source", "") for result in search_results]
        
        for k in top_k_values:
            if correct_source in sources[:k]:
                hits[k] += 1
                rank = sources.index(correct_source) + 1
                mrr[k] += 1 / rank if rank <= k else 0
        
        # Update progress bar postfix with current stats
        progress_bar.set_postfix({f'Hit@{max(top_k_values)}': f'{(hits[max(top_k_values)] / (progress_bar.n + 1)):.2f}'})
    
    # Close progress bar
    progress_bar.close()
    
    # Tính tỷ lệ điểm
    hit_ratios = {k: hits[k] / total_questions for k in top_k_values}
    mrr_scores = {k: mrr[k] / total_questions for k in top_k_values}
    
    # Ghi kết quả vào Excel với timestamp
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    output_file = f"evaluation_results_{timestamp}.xlsx"
    result_df = pd.DataFrame({
        'Hit@k': [f"Hit@{k}" for k in top_k_values],
        'Hit Ratio': [hit_ratios[k] for k in top_k_values],
        'MRR': [mrr_scores[k] for k in top_k_values]
    })
    
    result_df.to_excel(output_file, index=False)
    return hit_ratios, mrr_scores

# Chạy đánh giá
excel_path = excel  # Đường dẫn tới file Excel
print("Bắt đầu đánh giá...")
hit_ratios, mrr_scores = evaluate(excel_path)

# In kết quả
print("\nKết quả đánh giá:")
for k in sorted(hit_ratios.keys()):
    print(f"Hit@{k}: {hit_ratios[k]:.4f}")
    print(f"MRR@{k}: {mrr_scores[k]:.4f}")

Bắt đầu đánh giá...


Processing:  Theo số liệu tháng 01/2021, T...: 100%|██████████| 2560/2560 [10:14<00:00,  4.17it/s, Hit@20=0.76]


Kết quả đánh giá:
Hit@1: 0.3672
MRR@1: 0.3672
Hit@5: 0.6660
MRR@5: 0.4817
Hit@10: 0.7629
MRR@10: 0.4947
Hit@20: 0.7629
MRR@20: 0.4947


In [43]:
import json
import pandas as pd
from elasticsearch import Elasticsearch

# Hàm tìm kiếm
def search(query, top_k=1):
    embedding = embeddings.embed_query(query)  # Mã hóa truy vấn
    result = es.search(
        index=index,
        knn={
            "field": "vector",
            "query_vector": embedding,
            "k": top_k,
            "num_candidates": top_k * 2
        },
        source_excludes=["vector"]
    )
    res = [hit['_source']["metadata"]["source"] for hit in result['hits']['hits']]
    return res

# Kiểm tra hàm search
query = "Tôi muốn hỏi ban giám đốc đại học"
result = search(query, top_k=5)
print("Kết quả từ hàm search:", result)

chunk = pd.read_excel(excel)

# Lấy dòng đầu tiên
sample_row = chunk.iloc[0]
print("Dòng đầu tiên trong file Excel:")
print(f"Question: {sample_row['question']}")
print(f"Metadata: {sample_row['metadata']}")
print(f"Answer: {sample_row['answer']}")

# Kiểm tra và xử lý metadata
metadata = sample_row['metadata']
if isinstance(metadata, str):
    metadata = metadata.replace("'", "\"")
    try:
        metadata = json.loads(metadata)
    except json.JSONDecodeError as e:
        print("Lỗi khi giải mã JSON:", e)
        metadata = {}
print("Source trong metadata:", metadata.get("source", "Không có thông tin source"))


Kết quả từ hàm search: [{'source': 'Trung tâm Nghiên cứu Quốc tế về Trí tuệ nhân tạo.txt', 'id': '0'}, {'source': 'Đảng ủy Trường CNTT&TT.txt', 'id': '2'}, {'source': 'Đào tạo Đại học theo hình thức vừa làm vừa học.txt', 'id': '1'}, {'source': 'Quy định QLSV nước ngoài 2023.txt', 'id': '26'}, {'source': 'Hội đồng Đại học Bách khoa Hà Nội.txt', 'id': '7'}]
Dòng đầu tiên trong file Excel:
Question: Mục tiêu chiến lược phát triển Đại học Bách khoa Hà Nội giai đoạn 2017-2025 là gì?
Metadata: {'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '0'}
Answer: Mục tiêu chiến lược phát triển Đại học Bách khoa Hà Nội giai đoạn 2017-2025 là phát triển thành một đại học nghiên cứu đa lĩnh vực với nòng cốt là kỹ thuật và công nghệ, trong đó các đơn vị chuyên môn được tổ chức thành một số trường và khoa; viện và trung tâm nghiên cứu trực thuộc; đồng thời xây dựng hình mẫu thành công, phát triển bền vững của một đại học tự chủ toàn diện với mô hình quản trị tiên tiến, hệ thống tài c

In [12]:
from dotenv import load_dotenv
from langchain_elasticsearch import ElasticsearchRetriever
from model_config import VietnameseEmbeddings
from typing import Dict
import os
from retrieval import bm25_retriever, semantic_retriever, hybrid_retriever
# Configure logging with UTF-8 encoding
load_dotenv()
embeddings = VietnameseEmbeddings()
ELASTIC_URL = os.getenv("ELASTIC_URL")
api_key = os.getenv("api_key")

def hybrid_query(search_query: str, k_vector: int = 50, k_final: int = 5) -> Dict:

    # Chuyển câu hỏi thành vector embedding
    vector = embeddings.embed_query(search_query)
    
    return {
        "retriever": {
            "rrf": {
                "retrievers": [
                    {
                        "standard": {
                            "query": {
                                "match": {
                                    "text": search_query,
                                }
                            }
                        }
                    },
                    {
                        "knn": {
                            "field": "vector",
                            "query_vector": vector,
                            "k": k_vector,  # Vector search: tìm kiếm 50 kết quả
                            "num_candidates": k_vector * 2  # Số ứng viên xét là gấp đôi k_vector
                        }
                    },
                ],
            "rank_window_size": k_vector,
            "rank_constant": 60 #default = 60
            }
        }, "size": k_final # Rerank để lấy 5 kết quả tốt nhất
        
    }
def hybrid_retriever(index_name: str, top_k) -> ElasticsearchRetriever:
    return  ElasticsearchRetriever.from_es_params(
    index_name=index_name,
    body_func=hybrid_query(k_final=top_k),
    content_field="text",
    url=  ELASTIC_URL,
    api_key = api_key
)
def vectorqa_query(search_query: str ,top_k =1) -> Dict:
    vector = embeddings.embed_query(search_query)  # same embeddings as for indexing
    return {
        "knn": {
            "field": "vector",
            "query_vector": vector,
            "k": top_k,
            "num_candidates": 10,
        },
        "min_score": 0.85,
        #"source_excludes":["vector"],
    }

def qaretrieval(index_name: str) -> ElasticsearchRetriever:
    return  ElasticsearchRetriever.from_es_params(
    index_name=index_name,
    body_func=vectorqa_query,
    content_field="answer",
    url=  ELASTIC_URL,
    api_key = api_key
)
    
def bm25_query(search_query: str, top_k = 5) -> Dict:
    return {
        "query": {
            "match": {
                "text": search_query,
            },
        }, "size": top_k # Lấy 5 kết quả tốt nhất
    }

def bm25_retriever(index_name: str, top_k) -> ElasticsearchRetriever:
    return  ElasticsearchRetriever.from_es_params(
    index_name=index_name,
    body_func=bm25_query(top_k=top_k),
    content_field="text",
    url=  ELASTIC_URL,
    api_key = api_key
)
def semantic_query(search_query: str, top_k = 5) -> Dict:
    vector = embeddings.embed_query(search_query)  # same embeddings as for indexing
    return {
        "knn": {
            "field": "vector",
            "query_vector": vector,
            "k": top_k,
            "num_candidates": top_k * 2, # default = 10
        },
        #"min_score": 0.85,
        "source_excludes":["vector"],
    }
def semantic_retriever(index_name: str, top_k) -> ElasticsearchRetriever:
    return  ElasticsearchRetriever.from_es_params(
    index_name=index_name,
    body_func=semantic_query(top_k=top_k),
    content_field="text",
    url=  ELASTIC_URL,
    api_key = api_key
    )

In [13]:
bm25 = bm25_retriever(index_name=index, top_k=5)
semantic = semantic_retriever(index_name=index, top_k=5)
hybrid = hybrid_retriever(index_name=index,top_k=5)

In [15]:
query = "đại học bách khoa"
result = bm25.invoke(query)

In [18]:
result[0].metadata

{'_index': 'base',
 '_id': 'QP8hJZQBZ5S_PGgdmVEl',
 '_score': 8.635271,
 '_source': {'metadata': {'source': {'source': 'Câu lạc bộ sinh viên.txt',
    'id': '5'},
   'level': 0},
  'vector': [-0.03643405809998512,
   0.22395287454128265,
   -0.1976742148399353,
   -0.3647597134113312,
   -0.30882301926612854,
   0.09053202718496323,
   0.05811777338385582,
   0.42096975445747375,
   0.23024939000606537,
   0.0793197751045227,
   0.047040414065122604,
   -0.07203581184148788,
   0.35384735465049744,
   -0.3020235300064087,
   0.030449792742729187,
   -0.2954706847667694,
   0.35403943061828613,
   -0.17519789934158325,
   -0.30192503333091736,
   0.018747271969914436,
   -0.17155671119689941,
   0.062411461025476456,
   0.2093464732170105,
   0.045692138373851776,
   0.16983826458454132,
   0.3312515914440155,
   -0.09604984521865845,
   0.1886586993932724,
   -0.347994327545166,
   0.16060099005699158,
   0.18907418847084045,
   -0.09915850311517715,
   -0.02118634805083275,
   -0.3114